# Installation

In [ ]:
!apt install -y g++-10
!g++-10 --version
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-10 10
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-10 10
!update-alternatives --install /usr/bin/x86_64-linux-gnu-gcc gcc /usr/bin/gcc-10 10

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-10 is already the newest version (10.3.0-1ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
g++-10 (Ubuntu 10.3.0-1ubuntu1~20.04) 10.3.0
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

update-alternatives: renaming gcc link from /usr/bin/x86_64-linux-gnu-gcc to /usr/bin/gcc
update-alternatives: renaming gcc link from /usr/bin/gcc to /usr/bin/x86_64-linux-gnu-gcc


In [ ]:
!pip install optframe -v

def test_optframe():
  import optframe
  engine = optframe.Engine()
  engine.welcome()
  print("pyoptframe:", optframe.__version__)
#see: Execution Environment (Ambiente de Execução) -> See log registry (Ver registros de logs)
test_optframe()

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
pyoptframe: 5.0.21rc0


# Definition of Traveling Salesman Problem (TSP)

**Solution and ProblemContext**

In [ ]:
# OptFrame Python Demo TSP - Traveling Salesman Problem

import optframe

class SolutionTSP(object):
    def __init__(self):
        # number of cities in solution
        self.n = 0
        # visited cities as a list
        self.cities = []

    # MUST provide some printing mechanism
    def __str__(self):
        return f"SolutionTSP(n={self.n};cities={self.cities})"

    # MUST provide some deepcopy mechanism
    def __deepcopy__(self, memo):
        sol2 = SolutionTSP()
        sol2.n = self.n
        sol2.cities = [i for i in self.cities]
        return sol2

    def __del__(self):
        # print("~SolutionTSP")
        pass
import math

class ProblemContextTSP(object):
    def __init__(self):
        print('Init TSP')
        # may store current optframe engine for local usage
        self.engine = None
        # number of cities
        self.n = 0
        # x coordinates
        self.vx = []
        # y coordinates
        self.vy = []
        # distance matrix
        self.dist = []
        
   # Example: "3\n1 10 10\n2 20 20\n3 30 30\n"

    def load(self, filename):
        with open(filename, 'r') as f:
            lines = f.readlines()
            self.n = int(lines[0])
            for i in range(self.n):
               id_x_y = lines[i+1].split()
               # ignore id_x_y[0]
               self.vx.append(int(id_x_y[1]))
               self.vy.append(int(id_x_y[2]))
            #
            self.dist = [[0 for col in range(self.n)] for row in range(self.n)]
            for i in range(self.n):
               for j in range(self.n):
                  self.dist[i][j] = round(self.euclidean(self.vx[i], self.vy[i], self.vx[j], self.vy[j]))

    def euclidean(self, x1, y1, x2, y2):
        return math.sqrt((x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2))

    def __str__(self):
        return f"ProblemContextTSP(n={self.n};vx={self.vx};vy={self.vy};dist={self.dist})"



**Evaluator and Constructive**

In [ ]:

def mycallback_fevaluate(pTSP: ProblemContextTSP, s: SolutionTSP):
    assert (s.n == pTSP.n)
    assert (len(s.cities) == s.n)
    # remember this is an API1d method
    f = 0.0
    for i in range(pTSP.n-1):
      f += pTSP.dist[s.cities[i]][s.cities[i + 1]];
    f += pTSP.dist[s.cities[int(pTSP.n) - 1]][s.cities[0]];
    return f
import random

def mycallback_constructive(problemCtx: ProblemContextTSP) -> SolutionTSP:
    sol = SolutionTSP()
    for i in range(problemCtx.n):
        sol.cities.append(i)
    random.shuffle(sol.cities)
    sol.n = problemCtx.n
    return sol





**Definition of Move**

In [ ]:

# move
class MoveSwap(object):
    def __init__(self):
        #print('__init__ MoveSwap')
        self.i = 0
        self.j = 0

    def __del__(self):
        # print("~MoveSwap")
        pass

# Move Apply MUST return an Undo Move or Reverse Move (a Move that can undo current application)
def apply_swap(problemCtx: ProblemContextTSP, m: MoveSwap, sol: SolutionTSP) -> MoveSwap:
    i = m.i
    j = m.j
    #
    aux = sol.cities[j]
    sol.cities[j] = sol.cities[i]
    sol.cities[i] = aux
    # must create reverse move (j,i)
    mv = MoveSwap()
    mv.i = j
    mv.j = i
    return mv

# Moves can be applied or not (best performance is to have a True here)
def cba_swap(problemCtx: ProblemContextTSP, m: MoveSwap, sol: SolutionTSP) -> bool:
    return True

# Move equality must be provided
def eq_swap(problemCtx: ProblemContextTSP, m1: MoveSwap, m2: MoveSwap) -> bool:
    return (m1.i == m2.i) and (m1.j == m2.j)

**Definition of NS (random) and NSSeq (iterator)**

In [ ]:

def mycallback_ns_rand_swap(pTSP: ProblemContextTSP, sol: SolutionTSP) -> MoveSwap:
    i = random.randint(0, pTSP.n - 1)
    j = i
    while  j<= i:
        i = random.randint(0, pTSP.n - 1)
        j = random.randint(0, pTSP.n - 1)
    mv = MoveSwap()
    mv.i = i
    mv.j = j
    return mv


# For NSSeq, one must provide a Move Iterator
# A Move Iterator has five actions: Init, First, Next, IsDone and Current


class IteratorSwap(object):
    def __init__(self):
        # print('__init__ IteratorSwap')
        self.k = 0

    def __del__(self):
        # print("__del__ IteratorSwap")
        pass


def mycallback_nsseq_it_init_swap(pTSP: ProblemContextTSP, sol: SolutionTSP) -> IteratorSwap:
    it = IteratorSwap()
    it.i = -1
    it.j = -1
    return it


def mycallback_nsseq_it_first_swap(pTSP: ProblemContextTSP, it: IteratorSwap):
    it.i = 0
    it.j = 1


def mycallback_nsseq_it_next_swap(pTSP: ProblemContextTSP, it: IteratorSwap):
    if it.j < pTSP.n - 1:
        it.j = it.j+1
    else:
        it.i = it.i + 1
        it.j = it.i + 1

def mycallback_nsseq_it_isdone_swap(pTSP: ProblemContextTSP, it: IteratorSwap):
    return it.i >= pTSP.n - 1


def mycallback_nsseq_it_current_swap(pTSP: ProblemContextTSP, it: IteratorSwap):
    mv = MoveSwap()
    mv.i = it.i
    mv.j = it.j
    return mv

**Decoder for BRKGA**

In [ ]:
#
# random constructive: updates parameter ptr_array_double of type (LibArrayDouble*)
#
def mycallback_constructive_rk(problemCtx: ProblemContextTSP, ptr_array_double) -> int:
    rkeys = []
    for i in range(problemCtx.n):
        key = random.random() # [0,1] uniform
        rkeys.append(key)
    #
    ptr_array_double.contents.size = len(rkeys)
    ptr_array_double.contents.v = optframe.engine.callback_adapter_list_to_vecdouble(rkeys)
    return len(rkeys)

#
# decoder function: receives a problem instance and an array of random keys (as LibArrayDouble)
#
def mycallback_decoder_rk(problemCtx: ProblemContextTSP, array_double : optframe.engine.LibArrayDouble) -> SolutionTSP:
    #
    sol = SolutionTSP()
    #
    lpairs = []
    for i in range(array_double.size):
        p = [array_double.v[i], i]
        lpairs.append(p)
    #
    #print("lpairs: ", lpairs)
    sorted_list = sorted(lpairs)
    #print("sorted_list: ", sorted_list)
    #
    sol.n = problemCtx.n
    sol.cities = []
    for i in range(array_double.size):
        sol.cities.append(sorted_list[i][1]) # append index of city in order
    return sol

# BRKGA Example

In [ ]:
################
# BEGIN: MAIN
################

# CREATE EXAMPLE FOR FILE
f = open("tsp-example.txt", "w")
f.write("5\n")
f.write("1 10 10\n")
f.write("2 20 20\n")
f.write("3 30 30\n")
f.write("4 40 40\n")
f.write("5 50 50\n")
f.close()

In [ ]:

# set random seed for system
random.seed(0) # bad generator, just an example..

# loads problem from filesystem
pTSP = ProblemContextTSP()
pTSP.load('tsp-example.txt')

# initializes optframe engine
pTSP.engine = optframe.Engine(optframe.APILevel.API1d)
# print(pTSP)

# Register Basic Components

ev_idx = pTSP.engine.minimize(pTSP, mycallback_fevaluate)
print("evaluator id:", ev_idx)

c_rk_idx = pTSP.engine.add_constructive_rk(pTSP, mycallback_constructive_rk)
print("c_rk_idx=", c_rk_idx)

pTSP.engine.list_components("OptFrame:")

initepop_rk_id = pTSP.engine.build_component(
    "OptFrame:ComponentBuilder:EA:RK:BasicInitialEPopulationRKBuilder", 
    "OptFrame:Constructive:EA:RK:ConstructiveRK 0",
    "OptFrame:InitialEPopulation:EA:RK:InitialEPopulationRK")
print("initepop_rk_id=", initepop_rk_id)

print("")
print("WILL CREATE DECODER!!")
dec_rk_idx = pTSP.engine.add_decoder_rk(pTSP, mycallback_decoder_rk)
print("dec_rk_idx=", dec_rk_idx)

pTSP.engine.list_components("OptFrame:")

print("")
print("WILL BUILD COMPLETE DECODER WITH EVALUATOR!!")
drk_rk_id = pTSP.engine.build_component(
    "OptFrame:ComponentBuilder:EA:RK:BasicDecoderRandomKeysBuilder", 
    "OptFrame:GeneralEvaluator:Evaluator 0  OptFrame:EA:RK:DecoderRandomKeysNoEvaluation 0",
    "OptFrame:EA:RK:DecoderRandomKeys")
print("drk_rk_id=", drk_rk_id)


# =======================

print("")
print("testing builder (build_global_search) for BRKGA...")
print("")

g_idx = pTSP.engine.build_global_search(
    "OptFrame:ComponentBuilder:GlobalSearch:EA:RK:BRKGA",
    "OptFrame:EA:RK:DecoderRandomKeys 0  OptFrame:InitialEPopulation:EA:RK:InitialEPopulationRK 0 "
    "30 1000 0.4 0.3 0.6")
print("g_idx=", g_idx)



pTSP.engine.list_components("OptFrame:")

print("")
print("testing execution of GlobalSearch (run_global_search) for BRKGA...")
print("")

lout = pTSP.engine.run_global_search(g_idx, 3.0)
print('solution:', lout)

print("FINISHED")

Init TSP
evaluator id: 0
c_rk_idx= 0
initepop_rk_id= 0

WILL CREATE DECODER!!
dec_rk_idx= 0

WILL BUILD COMPLETE DECODER WITH EVALUATOR!!
drk_rk_id= 0

testing builder (build_global_search) for BRKGA...

g_idx= 0

testing execution of GlobalSearch (run_global_search) for BRKGA...

solution: SearchOutput(status=0;has_best=True;best_s=SolutionTSP(n=5;cities=[0, 1, 2, 4, 3]);best_e=112.0;)
FINISHED
